## 1. Robot and Map classes


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import time 

In [3]:
import sys 
sys.path.append('.\src')

from main import *



Thymio 1: (208.93, 166.0)
Thymio 2: (178.87, 166.0)
Goal_Center 1: (144.67, 70.0)
Thymio 1: (208.6, 165.6)
Thymio 2: (178.27, 165.3)
Goal_Center 1: (146.33, 70.0)
Thymio 1: (208.77, 165.5)
Thymio 2: (178.2, 165.2)
Green Object 1: (241.95, 77.95)
Green Object 2: (106.96, 69.05)
Green Object 3: (69.96, 228.92)
Green Object 4: (255.88, 230.88)
Invalid coordinate format: ('top_right', (241.95, 77.95))
Invalid coordinate format: ('top_left', (106.96, 69.05))
Invalid coordinate format: ('bottom_left', (69.96, 228.92))
Invalid coordinate format: ('bottom_right', (255.88, 230.88))
Goal_Center 1: (145.33, 70.0)
Thymio 1: (208.7, 165.67)
Thymio 2: (178.27, 165.27)
Goal_Center 1: (144.33, 70.0)
Thymio 1: (208.67, 165.67)
Thymio 2: (178.17, 165.33)
Goal_Center 1: (145.33, 70.0)
Thymio 1: (208.6, 165.53)
Thymio 2: (178.33, 165.3)
Green Object 1: (241.95, 77.95)
Green Object 2: (106.96, 68.96)
Green Object 3: (69.96, 228.96)
Green Object 4: (255.95, 230.95)
Invalid coordinate format: ('top_right', (

KeyboardInterrupt: 

In [3]:
from tdmclient import ClientAsync
client = ClientAsync()
node = await client.wait_for_node()
await node.lock()

In [5]:
test = []

test.append(([1,2],[3,4]))

#print first


TypeError: 'list' object is not callable

In [9]:
import cv2
import matplotlib.pyplot as plt


def generate_single_marker(aruco_dict):
   marker_size = int(input("Enter the marker size: "))
   marker_id = int(input("Enter the marker ID: "))

   marker_img = cv2.aruco.generateImageMarker(aruco_dict, marker_id,
    marker_size)

   cv2.imwrite("marker_{}.png".format(marker_id), marker_img)

   marker_img = cv2.imread("marker_{}.png".format(marker_id))

   cv2.imshow("Marker", marker_img)

   print("Dimensions:", marker_img.shape)

   cv2.waitKey(0)

def generate_bulk_markers(aruco_dict):
   marker_size = int(input("Enter the marker size: "))
   num_markers = int(input("Enter the number of markers to generate: "))
   marker_imgs = []

   for marker_id in range(num_markers):
       marker_img = cv2.aruco.generateImageMarker(aruco_dict, marker_id,marker_size)

       cv2.imwrite("marker_{}.png".format(marker_id), marker_img)
       marker_imgs.append(cv2.imread("marker_{}.png".format(marker_id)))

   for marker_img in marker_imgs:
       cv2.imshow("Marker", marker_img)
       print("Dimensions:", marker_img.shape)
       cv2.waitKey(0)

def main():
   aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)

   user_input = input("Press '1' to generate a single marker or "
                       "'2' to generate markers in bulk: ")

   if user_input == "1":
       generate_single_marker(aruco_dict)
   elif user_input == "2":
       generate_bulk_markers(aruco_dict)
   else:
       print("Invalid input. Please try again.")


main()



error: OpenCV(4.8.1) D:\a\opencv-python\opencv-python\opencv\modules\objdetect\src\aruco\aruco_dictionary.cpp:138: error: (-215:Assertion failed) sidePixels >= (markerSize + 2*borderBits) in function 'cv::aruco::Dictionary::generateImageMarker'
